In [1]:
import pandas as pd
import os
from sklearn.metrics import classification_report
from tqdm import tqdm
import itertools
import warnings
import sys
sys.path.append('../src')
import itertools

from models.classification_methods import get_classification_report
warnings.filterwarnings("ignore")


pd.set_option('display.max_columns', None)

In [2]:
def generate_latex_with_multirow_and_bold(df):
    latex_code = ''
    latex_code += '\\begin{table}[H]'
    latex_code += '\\scriptsize'
    latex_code += '\\centering'
    latex_code += "\\begin{tabular}{ll|rrrrrrr}\n\\toprule\n"
    latex_code += "input & classifier & Church & Bolsonaro & Hydrox. & Sinovac & Globo TV & Lula & overall \\\\ \n\\midrule\n"

    last_input = None
    multirow_count = 0

    for input_value in df['input'].unique():
        subset = df[df['input'] == input_value]
        max_overall_idx = subset['overall'].idxmax()

        for i, row in subset.iterrows():
            if row['input'] == last_input:
                latex_code += "& "
                multirow_count += 1
            else:
                if multirow_count > 0:
                    latex_code = latex_code.replace(f"multirow{{{multirow_count}}}", f"multirow{{{multirow_count + 1}}}", 1)
                if last_input is not None:
                    latex_code += "\\cmidrule(lr){1-9}\n"
                latex_code += f"\\multirow{{{1}}}{{*}}{{{row['input']}}} & "
                multirow_count = 1

            if i == max_overall_idx:
                row_data = [f"\\textbf{{{row[col]:.2f}}}" if col not in ['input', 'classifier'] else f"\\textbf{{{row[col]}}}" for col in df.columns[1:]]
                latex_code += " & ".join(row_data) + " \\\\ \n"
            else:
                latex_code += " & ".join([f"{row[col]:.2f}" if isinstance(row[col], float) else str(row[col]) for col in df.columns[1:]]) + " \\\\ \n"
            
            last_input = row['input']

    if multirow_count > 0:
        latex_code = latex_code.replace(f"multirow{{{multirow_count}}}", f"multirow{{{multirow_count + 1}}}", 1)

    latex_code += "\\cmidrule(lr){1-9}\n"
    latex_code += "\\bottomrule\n\\end{tabular}"
    latex_code += "\caption{F1 macro results}"
    latex_code += '\label{table:results_f1_macro}\n'
    latex_code += '\end{table}'

    return latex_code

In [3]:
test_results_path = '../reports/test_results/'

list_df_t = os.listdir(test_results_path)
list_df_t.sort()
list_df_t

['DummyClassifier_bo_concat_Texts_Timeline_concat_Texts_Timeline_test_results.csv',
 'DummyClassifier_bo_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_bo_users_Stance_test_results.csv',
 'DummyClassifier_bo_users_Timeline_test_results.csv',
 'DummyClassifier_cl_concat_Texts_Timeline_concat_Texts_Timeline_test_results.csv',
 'DummyClassifier_cl_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_cl_users_Stance_test_results.csv',
 'DummyClassifier_cl_users_Timeline_test_results.csv',
 'DummyClassifier_co_concat_Texts_Timeline_concat_Texts_Timeline_test_results.csv',
 'DummyClassifier_co_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_co_users_Stance_test_results.csv',
 'DummyClassifier_co_users_Timeline_test_results.csv',
 'DummyClassifier_gl_concat_Texts_Timeline_concat_Texts_Timeline_test_results.csv',
 'DummyClassifier_gl_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_gl_users_Stance_test_results.csv',
 'Dummy

In [4]:
# Target list
target_list = [
    'ig',
    'bo', 
    'cl', 
    'co', 
    'gl', 
    'lu'
]

dict_cp = {
    'cl':'Hydrox.',
    'lu':'Lula',
    'co':'Sinovac',
    'ig':'Church',
    'gl':'Globo TV',
    'bo':'Bolsonaro',
}

names = list(dict_cp.values())
names

['Hydrox.', 'Lula', 'Sinovac', 'Church', 'Globo TV', 'Bolsonaro']

## Create complete table

Get all the combinations of Stacking:

In [5]:
my_list = [
    'Stance','Texts', 'Timeline', 'Texts',
    'BoM', 'BoF', 'BoFr'
]

# Remove duplicados mantendo a ordem
unique_my_list = []
seen = set()
for item in my_list:
    if item not in seen:
        unique_my_list.append(item)
        seen.add(item)

all_combinations = []
for r in range(2, len(unique_my_list) + 1):
    all_combinations.extend(itertools.combinations(unique_my_list, r))



tuples_stacking_exps = []
clf_name = 'LogisticRegression'
preprocess_name = '-'
for comb in all_combinations:
    
    str_cols = "_".join(comb)
    
    exp_name = 'Stacking ' + " + ".join(comb)
    
    
    filename = 'Ensemble_LogisticRegression_{target}_' + str_cols + '_test_results.csv'
    tuples_stacking_exps.append((exp_name, preprocess_name, clf_name, filename))

In [6]:
tuples_stacking_exps

[('Stacking Stance + Texts',
  '-',
  'LogisticRegression',
  'Ensemble_LogisticRegression_{target}_Stance_Texts_test_results.csv'),
 ('Stacking Stance + Timeline',
  '-',
  'LogisticRegression',
  'Ensemble_LogisticRegression_{target}_Stance_Timeline_test_results.csv'),
 ('Stacking Stance + BoM',
  '-',
  'LogisticRegression',
  'Ensemble_LogisticRegression_{target}_Stance_BoM_test_results.csv'),
 ('Stacking Stance + BoF',
  '-',
  'LogisticRegression',
  'Ensemble_LogisticRegression_{target}_Stance_BoF_test_results.csv'),
 ('Stacking Stance + BoFr',
  '-',
  'LogisticRegression',
  'Ensemble_LogisticRegression_{target}_Stance_BoFr_test_results.csv'),
 ('Stacking Texts + Timeline',
  '-',
  'LogisticRegression',
  'Ensemble_LogisticRegression_{target}_Texts_Timeline_test_results.csv'),
 ('Stacking Texts + BoM',
  '-',
  'LogisticRegression',
  'Ensemble_LogisticRegression_{target}_Texts_BoM_test_results.csv'),
 ('Stacking Texts + BoF',
  '-',
  'LogisticRegression',
  'Ensemble_Logist

In [7]:

PATH_BEST_BoM = ""
PATH_BEST_BoF = ""

# (vectorizer,estimator, path_sring) 
results_tuples_stance = [
    # Stance
    ("Stance", "-" ,"dummy", "DummyClassifier_{target}_users_Stance_test_results.csv"),
    ("Stance", "tf-idf" ,"xgb", "XGBClassifier_TfidfVectorizer_{target}_users_Stance_test_results.csv"),
    ("Stance", "bertabaporu-base" ,"xgb", "bertimbau_xgb_{target}_users_emb_Stance_test_results.csv"),
    ("Stance", "-" ,"bertabaporu-base", "bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_Stance_test_results.csv"),
    ("Stance", "-",  "llama3:7b zero-shot", "llama3_{target}_Stance_prompt2_Stance_test_results.csv"),
    
    # Texts
    ("Texts", "-" ,"dummy", "DummyClassifier_{target}_top_mentioned_timelines_Texts_test_results.csv"),
    ("Texts", "tf-idf" ,"xgb", "XGBClassifier_TfidfVectorizer_{target}_top_mentioned_timelines_Texts_test_results.csv"),
    ("Texts", "bertabaporu-base" ,"xgb", "bertimbau_xgb_{target}_top_mentioned_timelines_emb_Texts_test_results.csv"),
    ("Texts", "-" ,"bertabaporu-base", "bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_Texts_test_results.csv"),
    ("Texts", "-" ,"bertabaporu-base (R)", "bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_scored_Texts_test_results.csv"),
    ("Texts", "-", "llama3:7b zero-shot [5] (R)", "llama3_{target}_filtered_Texts5_prompt2_Texts_test_results.csv"),
    ("Texts", "-", "llama3:7b zero-shot [10] (R)", "llama3_{target}_filtered_Texts10_prompt2_Texts_test_results.csv"),
    ("Texts", "-", "llama3:7b zero-shot [15] (R)", "llama3_{target}_filtered_Texts15_prompt2_Texts_test_results.csv"),
    
    # Timeline
    ("Timeline", "-" ,"dummy", "DummyClassifier_{target}_users_Timeline_test_results.csv"),
    ("Timeline", "tf-idf" ,"xgb", "XGBClassifier_TfidfVectorizer_{target}_users_Timeline_test_results.csv"),
    ("Timeline", "bertabaporu-base" ,"xgb", "bertimbau_xgb_{target}_users_emb_Timeline_test_results.csv"),
    ("Timeline", "-" ,"bertabaporu-base", "bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_Timeline_test_results.csv"),
    ("Timeline", "-" ,"bertabaporu-base (R)", "bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_scored_Timeline_test_results.csv"),
    ("Timeline", "-", "llama3:7b zero-shot [5] (R)", "llama3_{target}_filteredTimeline5_prompt2_Timeline_test_results.csv"),
    ("Timeline", "-", "llama3:7b zero-shot [10] (R)", "llama3_{target}_filteredTimeline10_prompt2_Timeline_test_results.csv"),
    ("Timeline", "-", "llama3:7b zero-shot [15] (R)", "llama3_{target}_filteredTimeline15_prompt2_Timeline_test_results.csv"),
    
    # Bag of xxx
    ('Bag of Friends', 'tf-idf', 'LogisticRegression', "TfidfVectorizer_SelectKBest_LogisticRegression_{target}_BagOfFriends_test_results.csv"),
    ('Bag of Followers', 'tf-idf', 'LogisticRegression', "TfidfVectorizer_SelectKBest_LogisticRegression_{target}_BagOfFollowers_test_results.csv"),
    ('Bag of Mentions', 'tf-idf', 'LogisticRegression', "TfidfVectorizer_SelectKBest_LogisticRegression_{target}_BagOfMentions_test_results.csv"),
    
    # Ensembles
    # ("Ensemble Texts + Timeline", "-", "LogisticRegression", "Ensemble_LogisticRegression_{target}_Texts_Timeline_test_results.csv"),
    # ("Ensemble Texts + Stance", "-", "LogisticRegression", "Ensemble_LogisticRegression_{target}_Texts_Stance_test_results.csv"),
    # ("Ensemble Stance + Timeline", "-", "LogisticRegression", "Ensemble_LogisticRegression_{target}_Stance_Timeline_test_results.csv"),
    # ("Ensemble Stance + Timeline + Texts", "-", "LogisticRegression", "Ensemble_LogisticRegression_{target}_Stance_Timeline_Texts_test_results.csv"),
    ("concat_Texts_Timeline", "tf-idf", "xgb", "XGBClassifier_TfidfVectorizer_{target}_concat_Texts_Timeline_concat_Texts_Timeline_test_results.csv")
]

In [8]:
results_tuples_stance = results_tuples_stance + tuples_stacking_exps

In [9]:
results_tuples_stance

[('Stance',
  '-',
  'dummy',
  'DummyClassifier_{target}_users_Stance_test_results.csv'),
 ('Stance',
  'tf-idf',
  'xgb',
  'XGBClassifier_TfidfVectorizer_{target}_users_Stance_test_results.csv'),
 ('Stance',
  'bertabaporu-base',
  'xgb',
  'bertimbau_xgb_{target}_users_emb_Stance_test_results.csv'),
 ('Stance',
  '-',
  'bertabaporu-base',
  'bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_Stance_test_results.csv'),
 ('Stance',
  '-',
  'llama3:7b zero-shot',
  'llama3_{target}_Stance_prompt2_Stance_test_results.csv'),
 ('Texts',
  '-',
  'dummy',
  'DummyClassifier_{target}_top_mentioned_timelines_Texts_test_results.csv'),
 ('Texts',
  'tf-idf',
  'xgb',
  'XGBClassifier_TfidfVectorizer_{target}_top_mentioned_timelines_Texts_test_results.csv'),
 ('Texts',
  'bertabaporu-base',
  'xgb',
  'bertimbau_xgb_{target}_top_mentioned_timelines_emb_Texts_test_results.csv'),
 ('Texts',
  '-',
  'bertabaporu-base',
  'bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_T

In [10]:


list_results = []
for text_col, vectorizer, estimator, path_results in results_tuples_stance:
    
    list_cr = []
    
    for target in target_list:
        
        
        path = test_results_path + path_results.format(target = target)
        df_results = pd.read_csv(path)
        df_results_or = df_results.copy()
        
        # get classification report df
        df_classification_report = get_classification_report(df_results.test, df_results.pred, cr_args = {})
        
        # create multindex
        column_indexes = [(metric,dict_cp[target]) for metric in df_classification_report.columns]
        multi_index_cols = pd.MultiIndex.from_tuples(column_indexes, names=['metric', 'target'])
        rows_indexes = [(text_col, vectorizer, estimator, cl) for cl in df_classification_report.index]
        multi_index_rows = pd.MultiIndex.from_tuples(rows_indexes, names=['text_col','vectorizer', 'estimator', 'class'])
        df_classification_report.columns = multi_index_cols
        df_classification_report.index = multi_index_rows
        
        # print(text_col, vectorizer, estimator,target)
        # print(path)
        # display(df_classification_report)
        
        list_cr.append(df_classification_report)
        
    df_results = pd.concat(list_cr, axis = 1)
    
    list_results.append(df_results)
    
df_results_final = pd.concat(list_results)

In [11]:
df_results_final

metric                                                                                        precision  \
target                                                                                           Church   
text_col                                           vectorizer estimator          class                    
Stance                                             -          dummy              against       0.565943   
                                                                                 accuracy      0.565943   
                                                                                 weighted avg  0.320292   
                                                                                 macro avg     0.282972   
                                                                                 for           0.000000   
...                                                                                                 ...   
Stacking Stance + Texts + Timeline + BoM + BoF ... -          LogisticRegression against       0.750000   
                                                                                 accuracy      0.731219   
                                                                                 weighted avg  0.729905   
                                                                                 macro avg     0.726852   
                                                                                 for           0.703704   

metric                                                                                           recall  \
target                                                                                           Church   
text_col                                           vectorizer estimator          class                    
Stance                                             -          dummy              against       1.000000   
                                                                                 accuracy      0.565943   
                                                                                 weighted avg  0.565943   
                                                                                 macro avg     0.500000   
                                                                                 for           0.000000   
...                                                                                                 ...   
Stacking Stance + Texts + Timeline + BoM + BoF ... -          LogisticRegression against       0.787611   
                                                                                 accuracy      0.731219   
                                                                                 weighted avg  0.731219   
                                                                                 macro avg     0.722651   
                                                                                 for           0.657692   

metric                                                                                         f1-score  \
target                                                                                           Church   
text_col                                           vectorizer estimator          class                    
Stance                                             -          dummy              against       0.722814   
                                                                                 accuracy      0.565943   
                                                                                 weighted avg  0.409072   
                                                                                 macro avg     0.361407   
                                                                                 for           0.000000   
...                                                                                                 ...   
Stacking Stance + Texts + Timeline + Bo

In [12]:
df_results_final.to_excel("../reports/table_complete_results.xlsx")

## Create table f1

In [13]:
mask_f1 = [True if  "f1-score" in col else False for col in df_results_final.columns]
mask_macro = [True if  "for" in col else False for col in df_results_final.index]

f1_df = df_results_final.loc[mask_macro,mask_f1]
f1_df[('f1-score','overall')] = f1_df.mean(axis=1)

f1_df

metric                                                                                         f1-score  \
target                                                                                           Church   
text_col                                           vectorizer       estimator           class             
Stance                                             -                dummy               for    0.000000   
                                                   tf-idf           xgb                 for    0.654762   
                                                   bertabaporu-base xgb                 for    0.824268   
                                                   -                bertabaporu-base    for    0.844622   
                                                                    llama3:7b zero-shot for    0.734426   
...                                                                                                 ...   
Stacking Stance + Texts + Timeline + BoF + BoFr    -                LogisticRegression  for    0.677228   
Stacking Stance + Texts + BoM + BoF + BoFr         -                LogisticRegression  for    0.814953   
Stacking Stance + Timeline + BoM + BoF + BoFr      -                LogisticRegression  for    0.734694   
Stacking Texts + Timeline + BoM + BoF + BoFr       -                LogisticRegression  for    0.650699   
Stacking Stance + Texts + Timeline + BoM + BoF ... -                LogisticRegression  for    0.679920   

metric                                                                                                   \
target                                                                                        Bolsonaro   
text_col                                           vectorizer       estimator           class             
Stance                                             -                dummy               for    0.000000   
                                                   tf-idf           xgb                 for    0.270270   
                                                   bertabaporu-base xgb                 for    0.324324   
                                                   -                bertabaporu-base    for    0.541667   
                                                                    llama3:7b zero-shot for    0.285714   
...                                                                                                 ...   
Stacking Stance + Texts + Timeline + BoF + BoFr    -                LogisticRegression  for    0.550000   
Stacking Stance + Texts + BoM + BoF + BoFr         -                LogisticRegression  for    0.604651   
Stacking Stance + Timeline + BoM + BoF + BoFr      -                LogisticRegression  for    0.550000   
Stacking Texts + Timeline + BoM + BoF + BoFr       -                LogisticRegression  for    0.512821   
Stacking Stance + Texts + Timeline + BoM + BoF ... -                LogisticRegression  for    0.550000   

metric                                                                                                   \
target                                                                                          Hydrox.   
text_col                                           vectorizer       estimator           class             
Stance                                             -                dummy               for    0.000000   
                                                   tf-idf           xgb                 for    0.719557   
                                                   bertabaporu-base xgb                 for    0.827709   
                                                   -                bertabaporu-base    for    0.854758   
                                                                    llama3:7b zero-shot for    0.652101   
...                                                                                                 ...   
Stacking Stance + Texts + Timeline + Bo

In [14]:
f1_df.sort_values([('f1-score','overall')])

metric                                                                      f1-score  \
target                                                                        Church   
text_col                             vectorizer estimator            class             
Stance                               -          dummy                for    0.000000   
Timeline                             -          dummy                for    0.000000   
Texts                                -          dummy                for    0.000000   
                                                bertabaporu-base     for    0.458937   
                                                bertabaporu-base (R) for    0.458234   
...                                                                              ...   
Stacking Stance + Texts + BoM + BoFr -          LogisticRegression   for    0.838095   
Stacking Stance + Texts + BoM        -          LogisticRegression   for    0.838095   
Stacking Stance + Texts + BoF + BoFr -          LogisticRegression   for    0.798521   
Stacking Stance + BoF + BoFr         -          LogisticRegression   for    0.844794   
Stacking Stance + BoF                -          LogisticRegression   for    0.830409   

metric                                                                                \
target                                                                     Bolsonaro   
text_col                             vectorizer estimator            class             
Stance                               -          dummy                for    0.000000   
Timeline                             -          dummy                for    0.000000   
Texts                                -          dummy                for    0.000000   
                                                bertabaporu-base     for    0.000000   
                                                bertabaporu-base (R) for    0.000000   
...                                                                              ...   
Stacking Stance + Texts + BoM + BoFr -          LogisticRegression   for    0.642857   
Stacking Stance + Texts + BoM        -          LogisticRegression   for    0.642857   
Stacking Stance + Texts + BoF + BoFr -          LogisticRegression   for    0.697674   
Stacking Stance + BoF + BoFr         -          LogisticRegression   for    0.666667   
Stacking Stance + BoF                -          LogisticRegression   for    0.711111   

metric                                                                                \
target                                                                       Hydrox.   
text_col                             vectorizer estimator            class             
Stance                               -          dummy                for    0.000000   
Timeline                             -          dummy                for    0.000000   
Texts                                -          dummy                for    0.000000   
                                                bertabaporu-base     for    0.218935   
                                                bertabaporu-base (R) for    0.536862   
...                                                                              ...   
Stacking Stance + Texts + BoM + BoFr -          LogisticRegression   for    0.877622   
Stacking Stance + Texts + BoM        -          LogisticRegression   for    0.877622   
Stacking Stance + Texts + BoF + BoFr -          LogisticRegression   for    0.879433   
Stacking Stance + BoF + BoFr         -          LogisticRegression   for    0.891986   
Stacking Stance + BoF                -          LogisticRegression   for    0.884283   

metric                                                                                \
target                                                                       Sinovac   
text_col                             vectorizer estimator            class             
Stance                               

## Create table f1 macro

In [15]:
mask_f1 = [True if  "f1-score" in col else False for col in df_results_final.columns]
mask_macro = [True if  "macro avg" in col else False for col in df_results_final.index]

f1_macro_df = df_results_final.loc[mask_macro,mask_f1]
f1_macro_df[('f1-score','overall')] = f1_macro_df.mean(axis=1)

f1_macro_df

metric                                                                                             f1-score  \
target                                                                                               Church   
text_col                                           vectorizer       estimator           class                 
Stance                                             -                dummy               macro avg  0.361407   
                                                   tf-idf           xgb                 macro avg  0.702021   
                                                   bertabaporu-base xgb                 macro avg  0.853801   
                                                   -                bertabaporu-base    macro avg  0.866276   
                                                                    llama3:7b zero-shot macro avg  0.729458   
...                                                                                                     ...   
Stacking Stance + Texts + Timeline + BoF + BoFr    -                LogisticRegression  macro avg  0.721009   
Stacking Stance + Texts + BoM + BoF + BoFr         -                LogisticRegression  macro avg  0.832816   
Stacking Stance + Timeline + BoM + BoF + BoFr      -                LogisticRegression  macro avg  0.775539   
Stacking Texts + Timeline + BoM + BoF + BoFr       -                LogisticRegression  macro avg  0.699811   
Stacking Stance + Texts + Timeline + BoM + BoF ... -                LogisticRegression  macro avg  0.724133   

metric                                                                                                       \
target                                                                                            Bolsonaro   
text_col                                           vectorizer       estimator           class                 
Stance                                             -                dummy               macro avg  0.462857   
                                                   tf-idf           xgb                 macro avg  0.595312   
                                                   bertabaporu-base xgb                 macro avg  0.625289   
                                                   -                bertabaporu-base    macro avg  0.737297   
                                                                    llama3:7b zero-shot macro avg  0.462857   
...                                                                                                     ...   
Stacking Stance + Texts + Timeline + BoF + BoFr    -                LogisticRegression  macro avg  0.748214   
Stacking Stance + Texts + BoM + BoF + BoFr         -                LogisticRegression  macro avg  0.776800   
Stacking Stance + Timeline + BoM + BoF + BoFr      -                LogisticRegression  macro avg  0.748214   
Stacking Texts + Timeline + BoM + BoF + BoFr       -                LogisticRegression  macro avg  0.728220   
Stacking Stance + Texts + Timeline + BoM + BoF ... -                LogisticRegression  macro avg  0.748214   

metric                                                                                                       \
target                                                                                              Hydrox.   
text_col                                           vectorizer       estimator           class                 
Stance                                             -                dummy               macro avg  0.334878   
                                                   tf-idf           xgb                 macro avg  0.734366   
                                                   bertabaporu-base xgb                 macro avg  0.830948   
                                                   -                bertabaporu-base    macro avg  0.848144   
                                                                    llama3:7b zero-shot macro avg  0.638889   
..

### Table for docs

In [16]:
f1_report = f1_macro_df.copy()
f1_report

metric                                                                                             f1-score  \
target                                                                                               Church   
text_col                                           vectorizer       estimator           class                 
Stance                                             -                dummy               macro avg  0.361407   
                                                   tf-idf           xgb                 macro avg  0.702021   
                                                   bertabaporu-base xgb                 macro avg  0.853801   
                                                   -                bertabaporu-base    macro avg  0.866276   
                                                                    llama3:7b zero-shot macro avg  0.729458   
...                                                                                                     ...   
Stacking Stance + Texts + Timeline + BoF + BoFr    -                LogisticRegression  macro avg  0.721009   
Stacking Stance + Texts + BoM + BoF + BoFr         -                LogisticRegression  macro avg  0.832816   
Stacking Stance + Timeline + BoM + BoF + BoFr      -                LogisticRegression  macro avg  0.775539   
Stacking Texts + Timeline + BoM + BoF + BoFr       -                LogisticRegression  macro avg  0.699811   
Stacking Stance + Texts + Timeline + BoM + BoF ... -                LogisticRegression  macro avg  0.724133   

metric                                                                                                       \
target                                                                                            Bolsonaro   
text_col                                           vectorizer       estimator           class                 
Stance                                             -                dummy               macro avg  0.462857   
                                                   tf-idf           xgb                 macro avg  0.595312   
                                                   bertabaporu-base xgb                 macro avg  0.625289   
                                                   -                bertabaporu-base    macro avg  0.737297   
                                                                    llama3:7b zero-shot macro avg  0.462857   
...                                                                                                     ...   
Stacking Stance + Texts + Timeline + BoF + BoFr    -                LogisticRegression  macro avg  0.748214   
Stacking Stance + Texts + BoM + BoF + BoFr         -                LogisticRegression  macro avg  0.776800   
Stacking Stance + Timeline + BoM + BoF + BoFr      -                LogisticRegression  macro avg  0.748214   
Stacking Texts + Timeline + BoM + BoF + BoFr       -                LogisticRegression  macro avg  0.728220   
Stacking Stance + Texts + Timeline + BoM + BoF ... -                LogisticRegression  macro avg  0.748214   

metric                                                                                                       \
target                                                                                              Hydrox.   
text_col                                           vectorizer       estimator           class                 
Stance                                             -                dummy               macro avg  0.334878   
                                                   tf-idf           xgb                 macro avg  0.734366   
                                                   bertabaporu-base xgb                 macro avg  0.830948   
                                                   -                bertabaporu-base    macro avg  0.848144   
                                                                    llama3:7b zero-shot macro avg  0.638889   
..

In [17]:
f1_report.reset_index(drop=False, inplace=True)
f1_report

metric                                           text_col        vectorizer  \
target                                                                        
0                                                  Stance                 -   
1                                                  Stance            tf-idf   
2                                                  Stance  bertabaporu-base   
3                                                  Stance                 -   
4                                                  Stance                 -   
..                                                    ...               ...   
77        Stacking Stance + Texts + Timeline + BoF + BoFr                 -   
78             Stacking Stance + Texts + BoM + BoF + BoFr                 -   
79          Stacking Stance + Timeline + BoM + BoF + BoFr                 -   
80           Stacking Texts + Timeline + BoM + BoF + BoFr                 -   
81      Stacking Stance + Texts + Timeline + BoM + BoF...                 -   

metric            estimator      class  f1-score                      \
target                                    Church Bolsonaro   Hydrox.   
0                     dummy  macro avg  0.361407  0.462857  0.334878   
1                       xgb  macro avg  0.702021  0.595312  0.734366   
2                       xgb  macro avg  0.853801  0.625289  0.830948   
3          bertabaporu-base  macro avg  0.866276  0.737297  0.848144   
4       llama3:7b zero-shot  macro avg  0.729458  0.462857  0.638889   
..                      ...        ...       ...       ...       ...   
77       LogisticRegression  macro avg  0.721009  0.748214  0.876297   
78       LogisticRegression  macro avg  0.832816  0.776800  0.886751   
79       LogisticRegression  macro avg  0.775539  0.748214  0.879788   
80       LogisticRegression  macro avg  0.699811  0.728220  0.879782   
81       LogisticRegression  macro avg  0.724133  0.748214  0.878043   

metric                                          
target   Sinovac  Globo TV      Lula   overall  
0       0.351759  0.372519  0.344578  0.371333  
1       0.752640  0.665243  0.654412  0.683999  
2       0.808036  0.781858  0.766993  0.777821  
3       0.834984  0.860252  0.789802  0.822792  
4       0.578779  0.770678  0.699381  0.646674  
..           ...       ...       ...       ...  
77      0.850709  0.631145  0.749336  0.762785  
78      0.860431  0.867255  0.804828  0.838147  
79      0.845545  0.797037  0.800781  0.807817  
80      0.858524  0.582406  0.749336  0.749680  
81      0.850709  0.626364  0.753113  0.763429  

[82 rows x 11 columns]

In [18]:
f1_report.columns

MultiIndex([(  'text_col',          ''),
            ('vectorizer',          ''),
            ( 'estimator',          ''),
            (     'class',          ''),
            (  'f1-score',    'Church'),
            (  'f1-score', 'Bolsonaro'),
            (  'f1-score',   'Hydrox.'),
            (  'f1-score',   'Sinovac'),
            (  'f1-score',  'Globo TV'),
            (  'f1-score',      'Lula'),
            (  'f1-score',   'overall')],
           names=['metric', 'target'])

In [19]:
new_columns = [col[0] if col[1] == '' else col[1] for col in f1_report.columns]
new_columns

['text_col',
 'vectorizer',
 'estimator',
 'class',
 'Church',
 'Bolsonaro',
 'Hydrox.',
 'Sinovac',
 'Globo TV',
 'Lula',
 'overall']

In [20]:
f1_report.columns = new_columns
f1_report

,text_col,vectorizer,estimator,class,Church,Bolsonaro,Hydrox.,Sinovac,Globo TV,Lula,overall
0,Stance,-,dummy,macro avg,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
1,Stance,tf-idf,xgb,macro avg,0.702021,0.595312,0.734366,0.752640,0.665243,0.654412,0.683999
2,Stance,bertabaporu-base,xgb,macro avg,0.853801,0.625289,0.830948,0.808036,0.781858,0.766993,0.777821
3,Stance,-,bertabaporu-base,macro avg,0.866276,0.737297,0.848144,0.834984,0.860252,0.789802,0.822792
4,Stance,-,llama3:7b zero-shot,macro avg,0.729458,0.462857,0.638889,0.578779,0.770678,0.699381,0.646674
...,...,...,...,...,...,...,...,...,...,...,...
77,Stacking Stance + Texts + Timeline + BoF + BoFr,-,LogisticRegression,macro avg,0.721009,0.748214,0.876297,0.850709,0.631145,0.749336,0.762785
78,Stacking Stance + Texts + BoM + BoF + BoFr,-,LogisticRegression,macro avg,0.832816,0.776800,0.886751,0.860431,0.867255,0.804828,0.838147
79,Stacking Stance + Timeline + BoM + BoF + BoFr,-,LogisticRegression,macro avg,0.775539,0.748214,0.879788,0.845545,0.797037,0.800781,0.807817
80,Stacking Texts + Timeline + BoM + BoF + BoFr,-,LogisticRegression,macro avg,0.699811,0.728220,0.879782,0.858524,0.582406,0.749336,0.749680


In [21]:
f1_report.drop(['class'],axis = 1, inplace=True)
f1_report

,text_col,vectorizer,estimator,Church,Bolsonaro,Hydrox.,Sinovac,Globo TV,Lula,overall
0,Stance,-,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
1,Stance,tf-idf,xgb,0.702021,0.595312,0.734366,0.752640,0.665243,0.654412,0.683999
2,Stance,bertabaporu-base,xgb,0.853801,0.625289,0.830948,0.808036,0.781858,0.766993,0.777821
3,Stance,-,bertabaporu-base,0.866276,0.737297,0.848144,0.834984,0.860252,0.789802,0.822792
4,Stance,-,llama3:7b zero-shot,0.729458,0.462857,0.638889,0.578779,0.770678,0.699381,0.646674
...,...,...,...,...,...,...,...,...,...,...
77,Stacking Stance + Texts + Timeline + BoF + BoFr,-,LogisticRegression,0.721009,0.748214,0.876297,0.850709,0.631145,0.749336,0.762785
78,Stacking Stance + Texts + BoM + BoF + BoFr,-,LogisticRegression,0.832816,0.776800,0.886751,0.860431,0.867255,0.804828,0.838147
79,Stacking Stance + Timeline + BoM + BoF + BoFr,-,LogisticRegression,0.775539,0.748214,0.879788,0.845545,0.797037,0.800781,0.807817
80,Stacking Texts + Timeline + BoM + BoF + BoFr,-,LogisticRegression,0.699811,0.728220,0.879782,0.858524,0.582406,0.749336,0.749680


In [22]:
f1_report.insert(
    1, 
    "classifier", 
    f1_report.apply(
        lambda x: f"{x['vectorizer']} + {x['estimator']}" if x['vectorizer'] != '-' else x['estimator'],
        axis = 1
        ).to_list()

)
f1_report.drop(['estimator', 'vectorizer'],axis =1, inplace = True)
f1_report

,text_col,classifier,Church,Bolsonaro,Hydrox.,Sinovac,Globo TV,Lula,overall
0,Stance,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
1,Stance,tf-idf + xgb,0.702021,0.595312,0.734366,0.752640,0.665243,0.654412,0.683999
2,Stance,bertabaporu-base + xgb,0.853801,0.625289,0.830948,0.808036,0.781858,0.766993,0.777821
3,Stance,bertabaporu-base,0.866276,0.737297,0.848144,0.834984,0.860252,0.789802,0.822792
4,Stance,llama3:7b zero-shot,0.729458,0.462857,0.638889,0.578779,0.770678,0.699381,0.646674
...,...,...,...,...,...,...,...,...,...
77,Stacking Stance + Texts + Timeline + BoF + BoFr,LogisticRegression,0.721009,0.748214,0.876297,0.850709,0.631145,0.749336,0.762785
78,Stacking Stance + Texts + BoM + BoF + BoFr,LogisticRegression,0.832816,0.776800,0.886751,0.860431,0.867255,0.804828,0.838147
79,Stacking Stance + Timeline + BoM + BoF + BoFr,LogisticRegression,0.775539,0.748214,0.879788,0.845545,0.797037,0.800781,0.807817
80,Stacking Texts + Timeline + BoM + BoF + BoFr,LogisticRegression,0.699811,0.728220,0.879782,0.858524,0.582406,0.749336,0.749680


In [23]:
f1_report.rename({"text_col":"input"}, axis = 1, inplace=True)
f1_report

,input,classifier,Church,Bolsonaro,Hydrox.,Sinovac,Globo TV,Lula,overall
0,Stance,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
1,Stance,tf-idf + xgb,0.702021,0.595312,0.734366,0.752640,0.665243,0.654412,0.683999
2,Stance,bertabaporu-base + xgb,0.853801,0.625289,0.830948,0.808036,0.781858,0.766993,0.777821
3,Stance,bertabaporu-base,0.866276,0.737297,0.848144,0.834984,0.860252,0.789802,0.822792
4,Stance,llama3:7b zero-shot,0.729458,0.462857,0.638889,0.578779,0.770678,0.699381,0.646674
...,...,...,...,...,...,...,...,...,...
77,Stacking Stance + Texts + Timeline + BoF + BoFr,LogisticRegression,0.721009,0.748214,0.876297,0.850709,0.631145,0.749336,0.762785
78,Stacking Stance + Texts + BoM + BoF + BoFr,LogisticRegression,0.832816,0.776800,0.886751,0.860431,0.867255,0.804828,0.838147
79,Stacking Stance + Timeline + BoM + BoF + BoFr,LogisticRegression,0.775539,0.748214,0.879788,0.845545,0.797037,0.800781,0.807817
80,Stacking Texts + Timeline + BoM + BoF + BoFr,LogisticRegression,0.699811,0.728220,0.879782,0.858524,0.582406,0.749336,0.749680


In [24]:
#f1_report.set_index(['input'],inplace=True)
#f1_report.drop('input', axis = 1,inplace=True)

In [25]:
f1_report.input.value_counts()

input
Texts                                                    8
Timeline                                                 8
Stance                                                   5
Stacking Texts + Timeline + BoM + BoF + BoFr             1
Stacking Stance + Timeline + BoM + BoF + BoFr            1
                                                        ..
Stacking Stance + Timeline + BoM                         1
Stacking Stance + Timeline + BoF                         1
Stacking Stance + Timeline + BoFr                        1
Stacking Stance + BoM + BoF                              1
Stacking Stance + Texts + Timeline + BoM + BoF + BoFr    1
Name: count, Length: 64, dtype: int64

In [26]:
f1_report

,input,classifier,Church,Bolsonaro,Hydrox.,Sinovac,Globo TV,Lula,overall
0,Stance,dummy,0.361407,0.462857,0.334878,0.351759,0.372519,0.344578,0.371333
1,Stance,tf-idf + xgb,0.702021,0.595312,0.734366,0.752640,0.665243,0.654412,0.683999
2,Stance,bertabaporu-base + xgb,0.853801,0.625289,0.830948,0.808036,0.781858,0.766993,0.777821
3,Stance,bertabaporu-base,0.866276,0.737297,0.848144,0.834984,0.860252,0.789802,0.822792
4,Stance,llama3:7b zero-shot,0.729458,0.462857,0.638889,0.578779,0.770678,0.699381,0.646674
...,...,...,...,...,...,...,...,...,...
77,Stacking Stance + Texts + Timeline + BoF + BoFr,LogisticRegression,0.721009,0.748214,0.876297,0.850709,0.631145,0.749336,0.762785
78,Stacking Stance + Texts + BoM + BoF + BoFr,LogisticRegression,0.832816,0.776800,0.886751,0.860431,0.867255,0.804828,0.838147
79,Stacking Stance + Timeline + BoM + BoF + BoFr,LogisticRegression,0.775539,0.748214,0.879788,0.845545,0.797037,0.800781,0.807817
80,Stacking Texts + Timeline + BoM + BoF + BoFr,LogisticRegression,0.699811,0.728220,0.879782,0.858524,0.582406,0.749336,0.749680


In [27]:
str_latex = generate_latex_with_multirow_and_bold(f1_report)

In [28]:
print(str_latex)

\begin{table}[H]\scriptsize\centering\begin{tabular}{ll|rrrrrrr}
\toprule
input & classifier & Church & Bolsonaro & Hydrox. & Sinovac & Globo TV & Lula & overall \\ 
\midrule
\multirow{2}{*}{Stance} & dummy & 0.36 & 0.46 & 0.33 & 0.35 & 0.37 & 0.34 & 0.37 \\ 
& tf-idf + xgb & 0.70 & 0.60 & 0.73 & 0.75 & 0.67 & 0.65 & 0.68 \\ 
& bertabaporu-base + xgb & 0.85 & 0.63 & 0.83 & 0.81 & 0.78 & 0.77 & 0.78 \\ 
& \textbf{bertabaporu-base} & \textbf{0.87} & \textbf{0.74} & \textbf{0.85} & \textbf{0.83} & \textbf{0.86} & \textbf{0.79} & \textbf{0.82} \\ 
& llama3:7b zero-shot & 0.73 & 0.46 & 0.64 & 0.58 & 0.77 & 0.70 & 0.65 \\ 
\cmidrule(lr){1-9}
\multirow{2}{*}{Texts} & dummy & 0.36 & 0.46 & 0.33 & 0.35 & 0.37 & 0.34 & 0.37 \\ 
& \textbf{tf-idf + xgb} & \textbf{0.60} & \textbf{0.50} & \textbf{0.60} & \textbf{0.67} & \textbf{0.55} & \textbf{0.58} & \textbf{0.58} \\ 
& bertabaporu-base + xgb & 0.59 & 0.50 & 0.61 & 0.66 & 0.53 & 0.57 & 0.58 \\ 
& bertabaporu-base & 0.59 & 0.46 & 0.45 & 0.61 & 0.51 